In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import sys

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    

print(sys.version)

3.8.1 (tags/v3.8.1:1b293b6, Dec 18 2019, 23:11:46) [MSC v.1916 64 bit (AMD64)]


In [2]:
# Read Data
account_assignment = pd.read_pickle('../data/pickle/Account_Assignment.pickle')

df = account_assignment
del account_assignment

df.head()

,BUSINESS_DIVISION,TIER,TERRITORY,TERRITORY_OWNER,TERRITORY_NUMBER,TERRITORY_TYPE,COUNTRY,STATE,ECRID,SIZE,ORGANIZATION_TYPE,LEVEL_15,LEVEL_14,LEVEL_13,LEVEL_12,CUSTOMER_NAME,PROVINCE
0,RS Corporate E&T accounts,Unassigned,CCSD-CP-NOAM-2,Sharon Malakoff,1441,Overlay,United States,Minnesota,ECR-1,Large (=Key Accounts),> $10B,SD-CORPORATE,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,ECR-1 - 3M Company,NaN
1,Corporate Research Solution,T1 - RF,CC-PS-BOOKS-APAC-MATURE-1,Delon Lee,1523,Overlay,India,MAHARASHTRA,ECR-378272,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-APAC,CCM-PS-BOOKS-APAC-1,CC-PS-BOOKS-APAC-MATURE-1,ECR-378272 - GlaxoSmithKline Pharmaceuticals Ltd,Maharashtra
2,RS Corporate PBT accounts,Unassigned,SSM-PS-RM-NOAM-RF-WEST-3,Mariam Willis,124063,Overlay,United States,California,ECR-378115,Medium (=Direct),Unknown,SD-PS-RM,RSSD-PS-RM-NOAM,RM-PS-RM-NOAM-W,SSM-PS-RM-NOAM-RF-WEST-3,ECR-378115 - Affymax Inc,SANTA CLARA
3,Corporate Research Solution,T1 - RF,SSMCC-PS-LS-EMEALA-EERUS-2,Karol Chomicki,1462,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-LS,RSSD-PS-LS-EMEALA,SSMCC-PS-LS-EMEALA-EERUS-2,SSMCC-PS-LS-EMEALA-EERUS-2,ECR-380044 - ABB Corporate Research,NaN
4,Corporate Research Solution,T1 - RF,SSMCC-PS-BOOKS-EMLA-2,Helena Paczuska,1426,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-EMLA,SSMCC-PS-BOOKS-EMLA-2,SSMCC-PS-BOOKS-EMLA-2,ECR-380044 - ABB Corporate Research,NaN


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown'], np.nan, inplace=True)
df

,BUSINESS_DIVISION,TIER,TERRITORY,TERRITORY_OWNER,TERRITORY_NUMBER,TERRITORY_TYPE,COUNTRY,STATE,ECRID,SIZE,ORGANIZATION_TYPE,LEVEL_15,LEVEL_14,LEVEL_13,LEVEL_12,CUSTOMER_NAME,PROVINCE
0,RS Corporate E&T accounts,Unassigned,CCSD-CP-NOAM-2,Sharon Malakoff,1441,Overlay,United States,Minnesota,ECR-1,Large (=Key Accounts),> $10B,SD-CORPORATE,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,ECR-1 - 3M Company,NaN
1,Corporate Research Solution,T1 - RF,CC-PS-BOOKS-APAC-MATURE-1,Delon Lee,1523,Overlay,India,MAHARASHTRA,ECR-378272,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-APAC,CCM-PS-BOOKS-APAC-1,CC-PS-BOOKS-APAC-MATURE-1,ECR-378272 - GlaxoSmithKline Pharmaceuticals Ltd,Maharashtra
2,RS Corporate PBT accounts,Unassigned,SSM-PS-RM-NOAM-RF-WEST-3,Mariam Willis,124063,Overlay,United States,California,ECR-378115,Medium (=Direct),NaN,SD-PS-RM,RSSD-PS-RM-NOAM,RM-PS-RM-NOAM-W,SSM-PS-RM-NOAM-RF-WEST-3,ECR-378115 - Affymax Inc,SANTA CLARA
3,Corporate Research Solution,T1 - RF,SSMCC-PS-LS-EMEALA-EERUS-2,Karol Chomicki,1462,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-LS,RSSD-PS-LS-EMEALA,SSMCC-PS-LS-EMEALA-EERUS-2,SSMCC-PS-LS-EMEALA-EERUS-2,ECR-380044 - ABB Corporate Research,NaN
4,Corporate Research Solution,T1 - RF,SSMCC-PS-BOOKS-EMLA-2,Helena Paczuska,1426,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-EMLA,SSMCC-PS-BOOKS-EMLA-2,SSMCC-PS-BOOKS-EMLA-2,ECR-380044 - ABB Corporate Research,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849143,A&G Research Solution,T4 - EF,CC-PS-ENG-APAC-ANZ-JPN-1,Su Min Lim,1505,Overlay,Japan,KANTO,ECR-34128846,Very Small (=Agents),Academic,SD-PS-ENG,RSSD-PS-ENG-APAC,CC-PS-ENG-APAC-ANZ-JPN-1,CC-PS-ENG-APAC-ANZ-JPN-1,ECR-34128846 - Professional Institute of Inter...,TOKYO
849144,A&G Research Solution,T4 - EF,SSM-PS-RM-ROAPAC-SEA-1,Wee Lim,1406,Overlay,Thailand,NaN,ECR-34293243,Small (=Telesales),Academic,SD-PS-RM,RSSD-PS-RM-ROAPAC,RM-PS-RM-SA-SEA-ANZ,SSM-PS-RM-ROAPAC-SEA-1,ECR-34293243 - Boromarajonani College of Nursi...,NaN
849145,A&G Research Solution,T4 - EF,CC-PS-RM-PURE-4,Adam Goh,84816,Overlay,India,DELHI,ECR-34297327,Medium (=Direct),Government,SD-PS-RM,RSSD-PS-RM-PURE,RM-PS-RM-PURE-RDM-APAC,CC-PS-RM-PURE-4,ECR-34297327 - National Project Implementation...,NaN
849146,A&G Research Solution,T4 - EF,CC-PS-RM-PURE-1,Kaveh Samadi Khah,1553,Overlay,France,PAYS DE LA LOIRE,ECR-34297388,Medium (=Direct),Academic,SD-PS-RM,RSSD-PS-RM-PURE,RM-PS-RM-PURE-RDM,CC-PS-RM-PURE-1,ECR-34297388 - Laval Regional Health/Social Tr...,NaN


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

A&G Research Solution               543963
Corporate Research Solution         205441
RS Corporate E&T accounts            49929
RS Corporate PBT accounts            22337
Health Solution                      12404
Clinical Solution                     9299
A&G Child accounts                    4303
Clinical Solution IDN Consortia        894
Clinical Solution IDN Member           524
RS Corporate unassigned accounts        54
Name: BUSINESS_DIVISION, dtype: int64
T4 - EF       433642
Unassigned    328016
T1 - RF        39570
T2 - RE        26550
T3 - ER        18130
Name: TIER, dtype: int64
CMD-PS-RM                         45173
SD-CP-NOAM-BOOKS-RIGHTS           45172
RSSD-PS-RM-PURE-HS                23165
CC-PS-BOOKS-APAC-MATURE-1         14846
CC-PS-BOOKS-NOAM-1                13041
                                  ...  
SD-A&G-APAC-CHINA                     3
RAD-A&G-APAC-ANZ                      2
SD-CP-GLOBAL-KA                       1
CMM-A&G-NOAM-GOV-GREAT-LAKES-1        1
RM

### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
16,PROVINCE,"[[nan, Maharashtra, SANTA CLARA, Gangnam-gu, K...",2294,object,58.78
10,ORGANIZATION_TYPE,"[[> $10B, nan, Medical/Health, > $1B-10B, Acad...",15,object,26.11
7,STATE,"[[Minnesota, MAHARASHTRA, California, nan, SEO...",912,object,17.70
9,SIZE,"[[Large (=Key Accounts), Medium (=Direct), nan...",6,object,1.43
8,ECRID,"[[ECR-1, ECR-378272, ECR-378115, ECR-380044, E...",68078,object,0.63
1,TIER,"[[Unassigned, T1 - RF, T2 - RE, T3 - ER, T4 - ...",6,object,0.38
5,TERRITORY_TYPE,"[[Overlay, RSS FLM, Prime, RSS CC Overlay, RSS...",6,object,0.00
6,COUNTRY,"[[United States, India, Poland, Korea, Republi...",196,object,0.00
4,TERRITORY_NUMBER,"[[1441, 1523, 124063, 1462, 1426, 124092, 1078...",903,int64,0.00
3,TERRITORY_OWNER,"[[Sharon Malakoff, Delon Lee, Mariam Willis, K...",483,object,0.00


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove other columns
df = df.drop(columns=[])

-------------------- Dropping Columns with missing data --------------------
Following 0 columns have missing data over the threshold and will be removed
Index([], dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
16,PROVINCE,"[[nan, Maharashtra, SANTA CLARA, Gangnam-gu, K...",2294,object,58.78
10,ORGANIZATION_TYPE,"[[> $10B, nan, Medical/Health, > $1B-10B, Acad...",15,object,26.11
7,STATE,"[[Minnesota, MAHARASHTRA, California, nan, SEO...",912,object,17.70
9,SIZE,"[[Large (=Key Accounts), Medium (=Direct), nan...",6,object,1.43
8,ECRID,"[[ECR-1, ECR-378272, ECR-378115, ECR-380044, E...",68078,object,0.63
1,TIER,"[[Unassigned, T1 - RF, T2 - RE, T3 - ER, T4 - ...",6,object,0.38
5,TERRITORY_TYPE,"[[Overlay, RSS FLM, Prime, RSS CC Overlay, RSS...",6,object,0.00
6,COUNTRY,"[[United States, India, Poland, Korea, Republi...",196,object,0.00
4,TERRITORY_NUMBER,"[[1441, 1523, 124063, 1462, 1426, 124092, 1078...",903,int64,0.00
3,TERRITORY_OWNER,"[[Sharon Malakoff, Delon Lee, Mariam Willis, K...",483,object,0.00


In [8]:
print(df.shape)
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

(849148, 17)
A&G Research Solution               543963
Corporate Research Solution         205441
RS Corporate E&T accounts            49929
RS Corporate PBT accounts            22337
Health Solution                      12404
Clinical Solution                     9299
A&G Child accounts                    4303
Clinical Solution IDN Consortia        894
Clinical Solution IDN Member           524
RS Corporate unassigned accounts        54
Name: BUSINESS_DIVISION, dtype: int64
T4 - EF       433642
Unassigned    328016
T1 - RF        39570
T2 - RE        26550
T3 - ER        18130
Name: TIER, dtype: int64
CMD-PS-RM                         45173
SD-CP-NOAM-BOOKS-RIGHTS           45172
RSSD-PS-RM-PURE-HS                23165
CC-PS-BOOKS-APAC-MATURE-1         14846
CC-PS-BOOKS-NOAM-1                13041
                                  ...  
SD-A&G-APAC-CHINA                     3
RAD-A&G-APAC-ANZ                      2
SD-CP-GLOBAL-KA                       1
CMM-A&G-NOAM-GOV-GREAT-LAKES-

### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

PROVINCE - Impute with most frequent

ORGANIZATION_TYPE - Impute with most frequent

STATE - Impute with most frequent

SIZE - Impute with ost frequent

In [9]:
# #Filling all missing values with 0
# # data = data.fillna(0)

# #Filling missing values with the most frequest values
# df['PROVINCE'].fillna(
#     df['PROVINCE'].value_counts().idxmax(), inplace=True)

# df['ORGANIZATION_TYPE'].fillna(
#     df['ORGANIZATION_TYPE'].value_counts().idxmax(), inplace=True)

# df['STATE'].fillna(
#     df['STATE'].value_counts().idxmax(), inplace=True)

# df['SIZE'].fillna(
#     df['SIZE'].value_counts().idxmax(), inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [10]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['TERRITORY_NUMBER'])

In [11]:
print(df.shape)
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

(849148, 17)
A&G Research Solution               543963
Corporate Research Solution         205441
RS Corporate E&T accounts            49929
RS Corporate PBT accounts            22337
Health Solution                      12404
Clinical Solution                     9299
A&G Child accounts                    4303
Clinical Solution IDN Consortia        894
Clinical Solution IDN Member           524
RS Corporate unassigned accounts        54
Name: BUSINESS_DIVISION, dtype: int64
T4 - EF       433642
Unassigned    328016
T1 - RF        39570
T2 - RE        26550
T3 - ER        18130
Name: TIER, dtype: int64
CMD-PS-RM                         45173
SD-CP-NOAM-BOOKS-RIGHTS           45172
RSSD-PS-RM-PURE-HS                23165
CC-PS-BOOKS-APAC-MATURE-1         14846
CC-PS-BOOKS-NOAM-1                13041
                                  ...  
SD-A&G-APAC-CHINA                     3
RAD-A&G-APAC-ANZ                      2
SD-CP-GLOBAL-KA                       1
CMM-A&G-NOAM-GOV-GREAT-LAKES-

# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
1,TIER,"[[Unassigned, T1 - RF, T2 - RE, T3 - ER, T4 - ...",6,object,0.38
5,TERRITORY_TYPE,"[[Overlay, RSS FLM, Prime, RSS CC Overlay, RSS...",6,object,0.00
9,SIZE,"[[Large (=Key Accounts), Medium (=Direct), nan...",6,object,1.43
0,BUSINESS_DIVISION,"[[RS Corporate E&T accounts, Corporate Researc...",10,object,0.00
11,LEVEL_15,"[[SD-CORPORATE, SD-PS-BOOKS, SD-PS-RM, SD-PS-L...",10,object,0.00
10,ORGANIZATION_TYPE,"[[> $10B, nan, Medical/Health, > $1B-10B, Acad...",15,object,26.11
12,LEVEL_14,"[[CCSD-CP-NOAM-2, RSSD-PS-BOOKS-APAC, RSSD-PS-...",61,object,0.00


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
1,TIER,"[[Unassigned, T1 - RF, T2 - RE, T3 - ER, T4 - ...",6,object,0.38
5,TERRITORY_TYPE,"[[Overlay, RSS FLM, Prime, RSS CC Overlay, RSS...",6,object,0.00
9,SIZE,"[[Large (=Key Accounts), Medium (=Direct), nan...",6,object,1.43
0,BUSINESS_DIVISION,"[[RS Corporate E&T accounts, Corporate Researc...",10,object,0.00
11,LEVEL_15,"[[SD-CORPORATE, SD-PS-BOOKS, SD-PS-RM, SD-PS-L...",10,object,0.00
10,ORGANIZATION_TYPE,"[[> $10B, nan, Medical/Health, > $1B-10B, Acad...",15,object,26.11
12,LEVEL_14,"[[CCSD-CP-NOAM-2, RSSD-PS-BOOKS-APAC, RSSD-PS-...",61,object,0.00


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 1 features - **********************
 TERRITORY_NUMBER standardized to TERRITORY_NUMBER


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
# print(columns_to_bin)

In [18]:
# # Specify Columns to encode
# columns_to_exclude = []
# columns_to_include = columns_to_bin
# columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
# columns_to_encode

In [19]:
# for column in columns_to_encode:
#     encoded_columns = pd.get_dummies(df[column])
#     print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
#     df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [20]:
# df.shape

In [21]:
df.head()

,BUSINESS_DIVISION,TIER,TERRITORY,TERRITORY_OWNER,TERRITORY_NUMBER,TERRITORY_TYPE,COUNTRY,STATE,ECRID,SIZE,ORGANIZATION_TYPE,LEVEL_15,LEVEL_14,LEVEL_13,LEVEL_12,CUSTOMER_NAME,PROVINCE
0,RS Corporate E&T accounts,Unassigned,CCSD-CP-NOAM-2,Sharon Malakoff,-1.19,Overlay,United States,Minnesota,ECR-1,Large (=Key Accounts),> $10B,SD-CORPORATE,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,ECR-1 - 3M Company,NaN
1,Corporate Research Solution,T1 - RF,CC-PS-BOOKS-APAC-MATURE-1,Delon Lee,-1.19,Overlay,India,MAHARASHTRA,ECR-378272,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-APAC,CCM-PS-BOOKS-APAC-1,CC-PS-BOOKS-APAC-MATURE-1,ECR-378272 - GlaxoSmithKline Pharmaceuticals Ltd,Maharashtra
2,RS Corporate PBT accounts,Unassigned,SSM-PS-RM-NOAM-RF-WEST-3,Mariam Willis,1.28,Overlay,United States,California,ECR-378115,Medium (=Direct),NaN,SD-PS-RM,RSSD-PS-RM-NOAM,RM-PS-RM-NOAM-W,SSM-PS-RM-NOAM-RF-WEST-3,ECR-378115 - Affymax Inc,SANTA CLARA
3,Corporate Research Solution,T1 - RF,SSMCC-PS-LS-EMEALA-EERUS-2,Karol Chomicki,-1.19,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-LS,RSSD-PS-LS-EMEALA,SSMCC-PS-LS-EMEALA-EERUS-2,SSMCC-PS-LS-EMEALA-EERUS-2,ECR-380044 - ABB Corporate Research,NaN
4,Corporate Research Solution,T1 - RF,SSMCC-PS-BOOKS-EMLA-2,Helena Paczuska,-1.19,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-EMLA,SSMCC-PS-BOOKS-EMLA-2,SSMCC-PS-BOOKS-EMLA-2,ECR-380044 - ABB Corporate Research,NaN


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [22]:
# convert date columns to date data type

date_columns = []

In [23]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [24]:
# calculate new variable activity_length

In [25]:
 df.head()

,BUSINESS_DIVISION,TIER,TERRITORY,TERRITORY_OWNER,TERRITORY_NUMBER,TERRITORY_TYPE,COUNTRY,STATE,ECRID,SIZE,ORGANIZATION_TYPE,LEVEL_15,LEVEL_14,LEVEL_13,LEVEL_12,CUSTOMER_NAME,PROVINCE
0,RS Corporate E&T accounts,Unassigned,CCSD-CP-NOAM-2,Sharon Malakoff,-1.19,Overlay,United States,Minnesota,ECR-1,Large (=Key Accounts),> $10B,SD-CORPORATE,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,CCSD-CP-NOAM-2,ECR-1 - 3M Company,NaN
1,Corporate Research Solution,T1 - RF,CC-PS-BOOKS-APAC-MATURE-1,Delon Lee,-1.19,Overlay,India,MAHARASHTRA,ECR-378272,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-APAC,CCM-PS-BOOKS-APAC-1,CC-PS-BOOKS-APAC-MATURE-1,ECR-378272 - GlaxoSmithKline Pharmaceuticals Ltd,Maharashtra
2,RS Corporate PBT accounts,Unassigned,SSM-PS-RM-NOAM-RF-WEST-3,Mariam Willis,1.28,Overlay,United States,California,ECR-378115,Medium (=Direct),NaN,SD-PS-RM,RSSD-PS-RM-NOAM,RM-PS-RM-NOAM-W,SSM-PS-RM-NOAM-RF-WEST-3,ECR-378115 - Affymax Inc,SANTA CLARA
3,Corporate Research Solution,T1 - RF,SSMCC-PS-LS-EMEALA-EERUS-2,Karol Chomicki,-1.19,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-LS,RSSD-PS-LS-EMEALA,SSMCC-PS-LS-EMEALA-EERUS-2,SSMCC-PS-LS-EMEALA-EERUS-2,ECR-380044 - ABB Corporate Research,NaN
4,Corporate Research Solution,T1 - RF,SSMCC-PS-BOOKS-EMLA-2,Helena Paczuska,-1.19,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-EMLA,SSMCC-PS-BOOKS-EMLA-2,SSMCC-PS-BOOKS-EMLA-2,ECR-380044 - ABB Corporate Research,NaN


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = ['TERRITORY_NUMBER', 'LEVEL_13', 'LEVEL_12', 'CUSTOMER_NAME']

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,BUSINESS_DIVISION,TIER,TERRITORY,TERRITORY_OWNER,TERRITORY_TYPE,COUNTRY,STATE,ECRID,SIZE,ORGANIZATION_TYPE,LEVEL_15,LEVEL_14,PROVINCE
0,RS Corporate E&T accounts,Unassigned,CCSD-CP-NOAM-2,Sharon Malakoff,Overlay,United States,Minnesota,ECR-1,Large (=Key Accounts),> $10B,SD-CORPORATE,CCSD-CP-NOAM-2,NaN
1,Corporate Research Solution,T1 - RF,CC-PS-BOOKS-APAC-MATURE-1,Delon Lee,Overlay,India,MAHARASHTRA,ECR-378272,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-APAC,Maharashtra
2,RS Corporate PBT accounts,Unassigned,SSM-PS-RM-NOAM-RF-WEST-3,Mariam Willis,Overlay,United States,California,ECR-378115,Medium (=Direct),NaN,SD-PS-RM,RSSD-PS-RM-NOAM,SANTA CLARA
3,Corporate Research Solution,T1 - RF,SSMCC-PS-LS-EMEALA-EERUS-2,Karol Chomicki,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-LS,RSSD-PS-LS-EMEALA,NaN
4,Corporate Research Solution,T1 - RF,SSMCC-PS-BOOKS-EMLA-2,Helena Paczuska,Overlay,Poland,NaN,ECR-380044,Medium (=Direct),> $10B,SD-PS-BOOKS,RSSD-PS-BOOKS-EMLA,NaN


Save clean pickle files

In [29]:
df.to_pickle("../"+cfg['clean_pickles_for_graphs_dir']+"/"+cfg['account_assignment_file']+"_for_graphs.pickle")